In [ ]:
%run -i code/utils.py

# 1. Reading Elevation File

In [ ]:
name_of_file = 'Data/chimmey_tops_dem_1m_utm17n.csv'
Mst = read_elevation(name_of_file)

# 2. Creating the FDS file
## 2.1 Defining the Region Where the Simulation Takes Place

In [ ]:
Min_x = 400
Max_x = 600
Min_y = 0
Max_y = 200
Mst = Mst[(Mst.x > Min_x-1) & (Mst.x < Max_x) & (Mst.y > Min_y-1) & (Mst.y < Max_y)]
Mst_o = Mst.copy()


# 2.2 Defining the Resolution of the Meshes and the obstacles in Meters

In [ ]:
R = 4
Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]
Ro = 4
Mst_o = Mst_o[(Mst_o.x%Ro==0) & (Mst_o.y%Ro==0)]

## 2.3 Calculating the Range of the Elevation 

In [ ]:
# Minimum and Maximum elevation
Min_z = math.floor(Mst['Elevation'].min())
Max_z = math.ceil(Mst['Elevation'].max())


# 2.4 Writing the FDS File 

In [ ]:
# Defining the time variables

T_begin  = 0.0
T_end    = 10.0
DT       = 0.1
Nmx      = 2    # Number of meshes in x-direction 
Nmy      = 2     # Number pf meshes in y-direction
Nmz      = 2     # Nmber of Meshes  in z-direction
PC       = 1
Location = [500,100] # Location of the fire
Child    = 'Region1'
write_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Location, Child)

# 3 Running the FDS program

## 3.1 Defining the folder where the fds input file is

In [ ]:
PATH =  os.getcwd()
Bin_folder = f"{PATH}/Bin"
FDS_FOLDER = f"FDSFiles/{foldername}"
os.chdir(FDS_FOLDER)

## 3.2 Running the fds command

In [ ]:
number_of_process = 4

In [ ]:
bash(['mpiexec', '-n', number_of_process,'fds',filename])

# 4 Reading the HRR Information from the plot3d Files

In [ ]:
os.chdir(PATH)
Number_of_meshes = Nmx*Nmy*Nmz

In [ ]:
reading_hrr(Child, Number_of_meshes)